# E-mail subject line filter

## Getting the data

In [2]:
%%bash
cd ../tmp/

export SPAM_ROOT="http://www.csmining.org/index.php"
export SPAM_PAGE="spam-assassin-datasets.html"
export SPAM_PAGE="$SPAM_PAGE?file=tl_files/Project_Datasets/SpamAssassin%20data"
export SPAM_SPAM_URL="$SPAM_PAGE/20050311_spam_2.tar.bz2"
export SPAM_HAM_URL="$SPAM_PAGE/20030228_easy_ham_2.tar.bz2"

wget -nc $SPAM_ROOT/$SPAM_SPAM_URL
wget -nc $SPAM_ROOT/$SPAM_HAM_URL

export SPAM_SPAM_NAME="${SPAM_SPAM_URL//\//%2F}"
export SPAM_HAM_NAME="${SPAM_HAM_URL//\//%2F}"

tar xf $SPAM_SPAM_NAME
tar xf $SPAM_HAM_NAME

El fichero “spam-assassin-datasets.html?file=tl_files%2FProject_Datasets%2FSpamAssassin%20data%2F20050311_spam_2.tar.bz2” ya está ahí, no se recupera.

El fichero “spam-assassin-datasets.html?file=tl_files%2FProject_Datasets%2FSpamAssassin%20data%2F20030228_easy_ham_2.tar.bz2” ya está ahí, no se recupera.



## Reading data

In [ ]:
import os

In [41]:
def list_words(text):
    words = []
    words_tmp = text.lower().split()
    for w in words_tmp:
        if w not in words and len(w) > 3:
            words.append(w)
    return words

In [40]:
files = os.listdir('../tmp/spam_2')

with open("../tmp/SubjectsSpam.out","w") as out:
    category = "spam"
    for fname in files:
        file_path = "../tmp/spam_2/" + fname

        data = None
        
        for encoding in ['ascii', 'utf-8', 'latin_1']:
            try:
                with open(file_path, encoding=encoding) as f:
                    data = f.readlines()
                    break
            except:
                pass
        
        if data is None:
            print('[EE] %s load failed.' % fname)
            continue
            
        for line in data:
            if line.startswith("Subject:"):
                line.replace(",", "")
                out.write("{0}, {1} \n".format(line[8:-1], category))
                break